In [1]:
import requests

In [2]:
def term_extraction(text,out,type):

  denotations=out['denotations']
  relavent_terms=[]
  
  if type=='drug':
    drug_terms=[]
    for i in denotations:
      if i['obj']== 'drug':
        drug_terms.append((i['span']['begin'],i['span']['end']))
   
    for i in drug_terms:
      start=i[0]
      end=i[1]
      relavent_terms.append(text[start:end])

  if type=='disease':
    species_terms=[]
    for i in denotations:
      if i['obj']== 'disease':
        species_terms.append((i['span']['begin'],i['span']['end']))
    
    for i in species_terms:
      start=i[0]
      end=i[1]
      relavent_terms.append(text[start:end])


  if type=='species':
    species_terms=[]
    for i in denotations:
      if i['obj']== 'species':
        species_terms.append((i['span']['begin'],i['span']['end']))
    
    for i in species_terms:
      start=i[0]
      end=i[1]
      relavent_terms.append(text[start:end])

  return(relavent_terms)


In [3]:
def query_raw(text, url="https://bern.korea.ac.kr/plain"):
  return requests.post(url, data={'sample_text': text}).json()


In [4]:
text = "X-rays were negative and physical assessment determined soft tissue damage to the lateral aspect of her ankle. She was initially treated with ice, an ace wrap, crutches and mild pain medications (Tylenol with codeine)"
text2 = "It is a skin disease causing much itchiness. Scratching leads to redness, swelling, cracking, weeping clear fluid, crusting, and scaling."
# out=query_raw(text)
# print(out)
# print(term_extraction(text,out,'drug'))
# print(term_extraction(text,out,'disease'))
# print(term_extraction(text,out,'species'))

In [5]:
import requests

def query_plain(text, url="http://bern2.korea.ac.kr/plain"):
    return requests.post(url, json={'text': text}).json()

if __name__ == '__main__':
    text = "Autophagy maintains tumour growth through circulating arginine."
    print(query_plain(text))

{'annotations': [{'id': ['mesh:D009369'], 'is_neural_normalized': False, 'mention': 'tumour', 'obj': 'disease', 'prob': 0.9999922513961792, 'span': {'begin': 20, 'end': 26}}, {'id': ['mesh:D001120'], 'is_neural_normalized': False, 'mention': 'arginine', 'obj': 'drug', 'prob': 0.9819279909133911, 'span': {'begin': 54, 'end': 62}}], 'text': 'Autophagy maintains tumour growth through circulating arginine.', 'timestamp': 'Thu Nov 10 23:16:05 +0000 2022'}


In [6]:
texty = ''
text = 'Autophagy maintains tumour growth through circulating arginine'

for o in range(2):

    texty= texty+f'{o}: '+text+f' :{o} '

In [7]:
texty

'0: Autophagy maintains tumour growth through circulating arginine :0 1: Autophagy maintains tumour growth through circulating arginine :1 '

In [8]:
import requests

def query_plain(text, url="http://bern2.korea.ac.kr/plain"):
    return requests.post(url, json={'text': text}).json()

if __name__ == '__main__':
    text = "Autophagy maintains tumour growth through circulating arginine"
#     print(query_plain(text))

In [9]:
text = "Autophagy maintains tumour growth through circulating arginine."
output = query_plain(texty)

In [10]:
import pandas as pd

In [11]:
df = pd.DataFrame(output['annotations'])

In [12]:
df.columns

Index(['id', 'is_neural_normalized', 'mention', 'obj', 'prob', 'span'], dtype='object')

In [13]:
texty

'0: Autophagy maintains tumour growth through circulating arginine :0 1: Autophagy maintains tumour growth through circulating arginine :1 '

In [14]:

numbers = re.findall('(\d:)',texty)

NameError: name 're' is not defined

In [15]:
import re
string = texty
pattern = '(\d:.*?):\d'


match=(re.search(pattern, string))

#getting the starting index using match.start()
print ("starting index", match.start())

#Getting the start and end index in tuple format using match.span()
print ("start and end index", match.span())


starting index 0
start and end index (0, 68)


In [25]:
import re
string = texty
pattern = '(\d:.*?):\d'

for o in re.findall('(\d:.*?):\d',texty):
    match=(re.search(o, string))
    #Getting the start and end index in tuple format using match.span()
    print ("start and end index", match.span())

start and end index (0, 66)
start and end index (69, 135)


'0: Autophagy maintains tumour growth through circulating arginine '

In [16]:
output

{'annotations': [{'id': ['mesh:D009369'],
   'is_neural_normalized': False,
   'mention': 'tumour',
   'obj': 'disease',
   'prob': 0.9999921321868896,
   'span': {'begin': 23, 'end': 29}},
  {'id': ['mesh:D001120'],
   'is_neural_normalized': False,
   'mention': 'arginine',
   'obj': 'drug',
   'prob': 0.9976105690002441,
   'span': {'begin': 57, 'end': 65}},
  {'id': ['mesh:D009369'],
   'is_neural_normalized': False,
   'mention': 'tumour',
   'obj': 'disease',
   'prob': 0.9999886751174927,
   'span': {'begin': 92, 'end': 98}},
  {'id': ['mesh:D001120'],
   'is_neural_normalized': False,
   'mention': 'arginine',
   'obj': 'drug',
   'prob': 0.9978335499763489,
   'span': {'begin': 126, 'end': 134}}],
 'text': '0: Autophagy maintains tumour growth through circulating arginine :0 1: Autophagy maintains tumour growth through circulating arginine :1',
 'timestamp': 'Thu Nov 10 23:16:16 +0000 2022'}

In [17]:
df

,id,is_neural_normalized,mention,obj,prob,span
0,[mesh:D009369],False,tumour,disease,0.999992,"{'begin': 23, 'end': 29}"
1,[mesh:D001120],False,arginine,drug,0.997611,"{'begin': 57, 'end': 65}"
2,[mesh:D009369],False,tumour,disease,0.999989,"{'begin': 92, 'end': 98}"
3,[mesh:D001120],False,arginine,drug,0.997834,"{'begin': 126, 'end': 134}"


In [66]:
x = df.annotations

AttributeError: 'DataFrame' object has no attribute 'annotations'

In [63]:
x

0    {'id': ['mesh:D009369'], 'is_neural_normalized...
1    {'id': ['mesh:D001120'], 'is_neural_normalized...
2    {'id': ['mesh:D009369'], 'is_neural_normalized...
3    {'id': ['mesh:D001120'], 'is_neural_normalized...
Name: annotations, dtype: object

In [60]:
x['mention']

'arginine'

In [61]:
x['obj']

'drug'